# AWS IoT関連のリソースを作る

In [ ]:

channel_name = 'channel-name'
pipeline_name = 'pipeline-name'
datastore_name = 'datastore-name'
train_dataset_name = 'dataset_train'
test_dataset_name = 'dataset_test'

In [ ]:
import boto3
iota_client = boto3.client('iotanalytics')


# チャネル作成

In [ ]:
response = iota_client.create_channel(
    channelName=channel_name,
    retentionPeriod={
        'unlimited': False,
        'numberOfDays': 1
    }
)
print(response)


# データストア作成

In [ ]:
response = iota_client.create_datastore(
    datastoreName=datastore_name,
    retentionPeriod={
        'unlimited': False,
        'numberOfDays': 1
    },
)
print(response)


# パイプライン作成

In [ ]:
datastore_activity_name = 'datastore_activity'
response = iota_client.create_pipeline(
    pipelineName=pipeline_name,
    pipelineActivities=[
        {
            'channel': {
                'name': 'channel_activity',
                'channelName': channel_name,
                'next': datastore_activity_name
            }
        },
        {
            'datastore': {
                'name': datastore_activity_name,
                'datastoreName': datastore_name
            },
        }
    ]
)
print(response)


# データセット作成

In [ ]:
bound_ts = '2013-12-10 00:00:00'

dataset_creation_params = [
    {
        'dataset_name': train_dataset_name,
        'action_name': 'train_query',
        'sql_condition': 'WHERE CAST("timestamp" AS TIMESTAMP) < TIMESTAMP \'{}\''.format(bound_ts)
    },{
        'dataset_name': test_dataset_name,
        'action_name': 'test_query',
        'sql_condition': 'WHERE CAST("timestamp" AS TIMESTAMP) >= TIMESTAMP \'{}\''.format(bound_ts)
    }
]


In [ ]:
for param_dic in dataset_creation_params:
    response = iota_client.create_dataset(
        datasetName=param_dic['dataset_name'],
        actions=[
            {
                'actionName': param_dic['action_name'],
                'queryAction': {
                    'sqlQuery': 'SELECT * FROM "{}" {}'.format(
                        datastore_name,
                        param_dic['sql_condition']
                    )
                },
            }
        ],
        retentionPeriod={
            'unlimited': False,
            'numberOfDays': 1
        }
    )
    
    print(response)